<a href="https://colab.research.google.com/github/marcosoares-92/IndustrialDataScienceWorkflow/blob/main/model_project/steel_industry_energy_consumption/etl_1_Agg_Manip_Timestamps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model project**

In this project, we will explore the Steel Industry Energy Consumption Dataset Data Set with the Industrial Data Science Workflow
- The full dataset description is available in the University of California, Irvine (UCI) Machine Learning Repository:

  https://archive.ics.uci.edu/ml/datasets/Steel+Industry+Energy+Consumption+Dataset

- This dataset contains real data from the DAEWOO Steel Co. Ltd in Gwangyang, South Korea, a smart small-scale steel industry.
- There are no missing values in the dataset.
- The dataset presents 35040 instances (rows) of 11 attributes (columns), containing numerical and categorical features.
- The response variable to be analyzed and optimized is the energy consumption.



# **Aggregation of Dataframes and Manipulation of Timestamps**

## _ETL Workflow Notebook 1_

## Content:
1. Applying a list of row filters;
2. Merging on timestamp;
3. Merging (joining) dataframes on given keys; and sorting the merged table;
4. Record linkage: fuzzy merging (joining) of dataframes on similar strings;
5. Concatenating (SQL Union/Stacking/Appending) dataframes;
6. Dataframe general characterization;
7. Dropping specific columns or rows from the dataframe;
8. Removing duplicate rows from the dataframe;
9. Removing all columns and rows that contain only missing values;
10. Grouping by timestamp;
11. Grouping by a given variable;
12. Extracting timestamp information;
13. Calculating differences between successive timestamps (delays);
14. Calculating timedeltas;
15. Adding or subtracting timedeltas;
16. Slicing the dataframe (selecting a specific subset of rows).

Marco Cesar Prado Soares, Data Scientist Specialist - Bayer Crop Science LATAM
- marcosoares.feq@gmail.com
- marco.soares@bayer.com

## **Load Python Libraries in Global Context**

In [ ]:
# Run installed library
import numpy as np
import pandas as pd
import idsw
from idsw import etl
from idsw.etl import etl_workflow as ewf

In [ ]:
# Run Python files
import numpy as np
import pandas as pd
import idsw
from etl import etl_workflow as ewf

## **Call the functions**

### **Mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [ ]:
SOURCE = 'google'
# SOURCE = 'google' for mounting the google drive;
# SOURCE = 'aws' for accessing an AWS S3 bucket

## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN SOURCE == 'aws':

PATH_TO_STORE_IMPORTED_S3_BUCKET = ''
# PATH_TO_STORE_IMPORTED_S3_BUCKET: path of the Python environment to which the
# S3 bucket contents will be imported. If it is None; or if it is an empty string; or if 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = '/', bucket will be imported to the root path. 
# Alternatively, input the path as a string (in quotes). e.g. 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = 'copied_s3_bucket'

S3_BUCKET_NAME = 'my_bucket'
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

S3_OBJECT_FOLDER_PREFIX = ""
# S3_OBJECT_FOLDER_PREFIX = None. Keep it None; or as an empty string 
# (S3_OBJECT_FOLDER_PREFIX = ''); or as the root "/" to import the 
# whole bucket content, instead of a single object from it.
# Alternatively, set it as a string containing the subfolder from the bucket to import:
# Suppose that your bucket (admin-created) has four objects with the following object 
# keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
# s3-dg.pdf. 
# The s3-dg.pdf key does not have a prefix, so its object appears directly 
# at the root level of the bucket. If you open the Development/ folder, you see 
# the Projects.xlsx object in it.
# In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
# where 'bucket' is the bucket's name, prefix = 'my_path/.../', without the
# 'file.csv' (file name with extension) last part.

# So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
# a given folder (directory) of the bucket.
# DO NOT PUT A SLASH before (to the right of) the prefix;
# DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

# Alternatively, provide the full path of a given file if you want to import only it:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
# where my_file is the file's name, and ext is its extension.


# Attention: after running this function for fetching AWS Simple Storage System (S3), 
# your 'AWS Access key ID' and your 'Secret access key' will be requested.
# The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
# other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
# and the prefix. All of these are sensitive information from the organization.
# Therefore, after importing the information, always remember of cleaning the output of this cell
# and of removing such information from the strings.
# Remember that these data may contain privilege for accessing protected information, 
# so it should not be used for non-authorized people.

# Also, remember of deleting the imported files from the workspace after finishing the analysis.
# The costs for storing the files in S3 is quite inferior than those for storing directly in the
# workspace. Also, files stored in S3 may be accessed for other users than those with access to
# the notebook's workspace.
idsw.mount_storage_system (source = SOURCE, path_to_store_imported_s3_bucket = PATH_TO_STORE_IMPORTED_S3_BUCKET, s3_bucket_name = S3_BUCKET_NAME, s3_obj_prefix = S3_OBJECT_FOLDER_PREFIX)

Associate the Python environment to your Google Drive account, and authorize the access in the opened window.
Mounted at /content/drive
Now your Python environment is connected to your Google Drive: the root directory of your environment is now the root of your Google Drive.
In Google Colab, navigate to the folder icon ('Files') of the left navigation menu to find a specific folder or file in your Google Drive.
Click on the folder or file name and select the elipsis (...) icon on the right of the name to reveal the option 'Copy path', which will give you the path to use as input for loading objects and files on your Python environment.
Caution: save your files into different directories of the Google Drive. If files are all saved in a same folder or directory, like the root path, they may not be accessible from your Python environment.
If you still cannot see the file after moving it to a different folder, reload the environment.


### **Importing the dataset**

In [ ]:
## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, xlsm, xlsb, odf, ods and odt), 
## JSON, txt, or CSV (comma separated values) files.

FILE_DIRECTORY_PATH = "/content/drive/MyDrive/model_project/steel_industry_energy_consumption"
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "" 
# or FILE_DIRECTORY_PATH = "folder"

FILE_NAME_WITH_EXTENSION = "steel_industry_data.csv"
# FILE_NAME_WITH_EXTENSION - (string, in quotes): input the name of the file with the 
# extension. e.g. FILE_NAME_WITH_EXTENSION = "file.xlsx", or, 
# FILE_NAME_WITH_EXTENSION = "file.csv", "file.txt", or "file.json"
# Again, the extensions may be: xls, xlsx, xlsm, xlsb, odf, ods, odt, json, txt or csv.

LOAD_TXT_FILE_WITH_JSON_FORMAT = False
# LOAD_TXT_FILE_WITH_JSON_FORMAT = False. Set LOAD_TXT_FILE_WITH_JSON_FORMAT = True 
# if you want to read a file with txt extension containing a text formatted as JSON 
# (but not saved as JSON).
# WARNING: if LOAD_TXT_FILE_WITH_JSON_FORMAT = True, all the JSON file parameters of the 
# function (below) must be set. If not, an error message will be raised.

HOW_MISSING_VALUES_ARE_REGISTERED = None
# HOW_MISSING_VALUES_ARE_REGISTERED = None: keep it None if missing values are registered as None,
# empty or np.nan. Pandas automatically converts None to NumPy np.nan objects (floats).
# This parameter manipulates the argument na_values (default: None) from Pandas functions.
# By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, 
#‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, 
# ‘n/a’, ‘nan’, ‘null’.

# If a different denomination is used, indicate it as a string. e.g.
# HOW_MISSING_VALUES_ARE_REGISTERED = '.' will convert all strings '.' to missing values;
# HOW_MISSING_VALUES_ARE_REGISTERED = 0 will convert zeros to missing values.

# If dict passed, specific per-column NA values. For example, if zero is the missing value
# only in column 'numeric_col', you can specify the following dictionary:
# how_missing_values_are_registered = {'numeric-col': 0}

    
HAS_HEADER = True
# HAS_HEADER = True if the the imported table has headers (row with columns names).
# Alternatively, HAS_HEADER = False if the dataframe does not have header.

DECIMAL_SEPARATOR = '.'
# DECIMAL_SEPARATOR = '.' - String. Keep it '.' or None to use the period ('.') as
# the decimal separator. Alternatively, specify here the separator.
# e.g. DECIMAL_SEPARATOR = ',' will set the comma as the separator.
# It manipulates the argument 'decimal' from Pandas functions.

TXT_CSV_COL_SEP = "comma"
# txt_csv_col_sep = "comma" - This parameter has effect only when the file is a 'txt'
# or 'csv'. It informs how the different columns are separated.
# Alternatively, txt_csv_col_sep = "comma", or txt_csv_col_sep = "," 
# for columns separated by comma;
# txt_csv_col_sep = "whitespace", or txt_csv_col_sep = " " 
# for columns separated by simple spaces.
# You can also set a specific separator as string. For example:
# txt_csv_col_sep = '\s+'; or txt_csv_col_sep = '\t' (in this last example, the tabulation
# is used as separator for the columns - '\t' represents the tab character).

## Parameters for loading Excel files:

LOAD_ALL_SHEETS_AT_ONCE = False
# LOAD_ALL_SHEETS_AT_ONCE = False - This parameter has effect only when for Excel files.
# If LOAD_ALL_SHEETS_AT_ONCE = True, the function will return a list of dictionaries, each
# dictionary containing 2 key-value pairs: the first key will be 'sheet', and its
# value will be the name (or number) of the table (sheet). The second key will be 'df',
# and its value will be the pandas dataframe object obtained from that sheet.
# This argument has preference over SHEET_TO_LOAD. If it is True, all sheets will be loaded.
    
SHEET_TO_LOAD = None
# SHEET_TO_LOAD - This parameter has effect only when for Excel files.
# keep SHEET_TO_LOAD = None not to specify a sheet of the file, so that the first sheet
# will be loaded.
# SHEET_TO_LOAD may be an integer or an string (inside quotes). SHEET_TO_LOAD = 0
# loads the first sheet (sheet with index 0); SHEET_TO_LOAD = 1 loads the second sheet
# of the file (index 1); SHEET_TO_LOAD = "Sheet1" loads a sheet named as "Sheet1".
# Declare a number to load the sheet with that index, starting from 0; or declare a
# name to load the sheet with that name.

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFIX_LIST = None
# JSON_METADATA_PREFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFIX_LIST = ['name', 'last']


# The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = idsw.load_pandas_dataframe (file_directory_path = FILE_DIRECTORY_PATH, file_name_with_extension = FILE_NAME_WITH_EXTENSION, load_txt_file_with_json_format = LOAD_TXT_FILE_WITH_JSON_FORMAT, how_missing_values_are_registered = HOW_MISSING_VALUES_ARE_REGISTERED, has_header = HAS_HEADER, decimal_separator = DECIMAL_SEPARATOR, txt_csv_col_sep = TXT_CSV_COL_SEP, load_all_sheets_at_once = LOAD_ALL_SHEETS_AT_ONCE, sheet_to_load = SHEET_TO_LOAD, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_prefix_list = JSON_METADATA_PREFIX_LIST)

# OBS: If an Excel file is loaded and LOAD_ALL_SHEETS_AT_ONCE = True, then the object
# dataset will be a list of dictionaries, with 'sheet' as key containing the sheet name; and 'df'
# as key correspondent to the Pandas dataframe. So, to access the 3rd dataframe (index 2, since
# indexing starts from zero): df = dataframe[2]['df'], where dataframe is the list returned.

Tokenization took: 30.87 ms
Type conversion took: 29.17 ms
Parser memory cleanup took: 0.01 ms
Dataset extracted from /content/drive/MyDrive/model_project/steel_industry_energy_consumption/steel_industry_data.csv. Check the 10 first rows of this dataframe:



,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load
5,01/01/2018 01:30,3.28,3.56,0.0,0.0,67.76,100.0,5400,Weekday,Monday,Light_Load
6,01/01/2018 01:45,3.60,4.14,0.0,0.0,65.62,100.0,6300,Weekday,Monday,Light_Load
7,01/01/2018 02:00,3.60,4.28,0.0,0.0,64.37,100.0,7200,Weekday,Monday,Light_Load
8,01/01/2018 02:15,3.28,3.64,0.0,0.0,66.94,100.0,8100,Weekday,Monday,Light_Load
9,01/01/2018 02:30,3.78,4.72,0.0,0.0,62.51,100.0,9000,Weekday,Monday,Light_Load


### **Characterizing the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

#New dataframes saved as df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values.
# Simply modify this object on the left of equality:
df_shape, df_columns_array, df_dtypes, df_general_statistics, df_missing_values = ewf.df_general_characterization (df = DATASET)

Dataframe's 10 first rows:



,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load
5,01/01/2018 01:30,3.28,3.56,0.0,0.0,67.76,100.0,5400,Weekday,Monday,Light_Load
6,01/01/2018 01:45,3.60,4.14,0.0,0.0,65.62,100.0,6300,Weekday,Monday,Light_Load
7,01/01/2018 02:00,3.60,4.28,0.0,0.0,64.37,100.0,7200,Weekday,Monday,Light_Load
8,01/01/2018 02:15,3.28,3.64,0.0,0.0,66.94,100.0,8100,Weekday,Monday,Light_Load
9,01/01/2018 02:30,3.78,4.72,0.0,0.0,62.51,100.0,9000,Weekday,Monday,Light_Load




Dataframe's 10 last rows:



,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
35030,31/12/2018 21:45,3.42,0.00,13.36,0.0,100.00,24.80,78300,Weekday,Monday,Light_Load
35031,31/12/2018 22:00,3.42,0.00,13.07,0.0,100.00,25.31,79200,Weekday,Monday,Light_Load
35032,31/12/2018 22:15,3.24,3.38,0.22,0.0,69.20,99.77,80100,Weekday,Monday,Light_Load
35033,31/12/2018 22:30,3.67,4.43,0.00,0.0,63.80,100.00,81000,Weekday,Monday,Light_Load
35034,31/12/2018 22:45,3.82,4.54,0.00,0.0,64.38,100.00,81900,Weekday,Monday,Light_Load
35035,31/12/2018 23:00,3.85,4.86,0.00,0.0,62.10,100.00,82800,Weekday,Monday,Light_Load
35036,31/12/2018 23:15,3.74,3.74,0.00,0.0,70.71,100.00,83700,Weekday,Monday,Light_Load
35037,31/12/2018 23:30,3.78,3.17,0.07,0.0,76.62,99.98,84600,Weekday,Monday,Light_Load
35038,31/12/2018 23:45,3.78,3.06,0.11,0.0,77.72,99.96,85500,Weekday,Monday,Light_Load
35039,31/12/2018 00:00,3.67,3.02,0.07,0.0,77.22,99.98,0,Weekday,Monday,Light_Load




Dataframe's shape = (number of rows, number of columns) =



...

dataframe_column
date                                     object
Usage_kWh                               float64
Lagging_Current_Reactive.Power_kVarh    float64
Leading_Current_Reactive_Power_kVarh    float64
CO2(tCO2)                               float64
Lagging_Current_Power_Factor            float64
Leading_Current_Power_Factor            float64
NSM                                       int64
WeekStatus                               object
Day_of_week                              object
Load_Type                                object
Name: dtype_series, dtype: object



Dataframe's general (summary) statistics for numeric variables:



dataframe_column,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,27.386892,13.035384,3.870949,0.011524,80.578056,84.367870,42750.000000
std,33.444380,16.306000,7.424463,0.016151,18.921322,30.456535,24940.534317
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,2.300000,0.000000,0.000000,63.320000,99.700000,21375.000000
50%,4.570000,5.000000,0.000000,0.000000,87.960000,100.000000,42750.000000
75%,51.237500,22.640000,2.090000,0.020000,99.022500,100.000000,64125.000000
max,157.180000,96.910000,27.760000,0.070000,100.000000,100.000000,85500.000000




Missing values on each feature; and missingness considering all rows from the dataframe:
(note: 'missingness_accross_rows' was calculated by: checking which rows have at least one missing value (NA); and then comparing total rows with NAs with total rows in the dataframe).



,count_of_missing_values,proportion_of_missing_values,percent_of_missing_values
dataframe_column,,,
date,0,0.0,0.0
Usage_kWh,0,0.0,0.0
Lagging_Current_Reactive.Power_kVarh,0,0.0,0.0
Leading_Current_Reactive_Power_kVarh,0,0.0,0.0
CO2(tCO2),0,0.0,0.0
Lagging_Current_Power_Factor,0,0.0,0.0
Leading_Current_Power_Factor,0,0.0,0.0
NSM,0,0.0,0.0
WeekStatus,0,0.0,0.0


### **Removing duplicate rows from the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

LIST_OF_COLUMNS_TO_ANALYZE = None
# if LIST_OF_COLUMNS_TO_ANALYZE = None, the whole dataset will be analyzed, i.e., rows
# will be removed only if they have same values for all columns from the dataset.
# Alternatively, pass a list of columns names (strings), if you want to remove rows with
# same values for that combination of columns. Pass it as a list, even if there is a single column
# being declared.
# e.g. LIST_OF_COLUMNS_TO_ANALYZE = ['column1'] will check only 'column1'. Entries with same value
# on 'column1' will be considered duplicates and will be removed.
# LIST_OF_COLUMNS_TO_ANALYZE = ['col1', 'col2',  'col3'] will analyze the combination of 3 columns:
# 'col1', 'col2', and 'col3'. Only rows with same value for these 3 columns will be considered
# duplicates and will be removed.

WHICH_ROW_TO_KEEP = 'first'
# WHICH_ROW_TO_KEEP = 'first' will keep the first detected row and remove all other duplicates. If
# None or an invalid string is input, this method will be selected.
# WHICH_ROW_TO_KEEP = 'last' will keep only the last detected duplicate row, and remove all the others.
    
RESET_INDEX_AFTER_DROP = True
# RESET_INDEX_AFTER_DROP = True. keep it True to restarting the indexing numeration after dropping.
# Alternatively, set RESET_INDEX_AFTER_DROP = False to keep the original numeration (the removed indices
# will be missing).

# New dataframe saved as cleaned_df. Simply modify this object on the left of equality:
dataset = ewf.remove_duplicate_rows (df = DATASET, list_of_columns_to_analyze = LIST_OF_COLUMNS_TO_ANALYZE, which_row_to_keep = WHICH_ROW_TO_KEEP, reset_index_after_drop = RESET_INDEX_AFTER_DROP)

The rows with duplicate entries were successfully removed.
Only the first one of the duplicate entries was kept in the dataset.

The indices of the dataset were successfully restarted.

Check the 10 first rows from the returned dataset:



,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load
5,01/01/2018 01:30,3.28,3.56,0.0,0.0,67.76,100.0,5400,Weekday,Monday,Light_Load
6,01/01/2018 01:45,3.60,4.14,0.0,0.0,65.62,100.0,6300,Weekday,Monday,Light_Load
7,01/01/2018 02:00,3.60,4.28,0.0,0.0,64.37,100.0,7200,Weekday,Monday,Light_Load
8,01/01/2018 02:15,3.28,3.64,0.0,0.0,66.94,100.0,8100,Weekday,Monday,Light_Load
9,01/01/2018 02:30,3.78,4.72,0.0,0.0,62.51,100.0,9000,Weekday,Monday,Light_Load


### **Removing all columns and rows that contain only missing values**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

LIST_OF_COLUMNS_TO_IGNORE = None
# list_of_columns_to_ignore: if you do not want to check a specific column, pass its name
# (header) as an element from this list. It should be declared as a list even if it contains
# a single value.
# e.g. list_of_columns_to_ignore = ['column1'] will not analyze missing values in column named
# 'column1'; list_of_columns_to_ignore = ['col1', 'col2'] will ignore columns 'col1' and 'col2'

# Cleaned dataframe returned as cleaned_df. 
# Simply modify this object on the left of equality:
dataset = ewf.remove_completely_blank_rows_and_columns (df = DATASET, list_of_columns_to_ignore = LIST_OF_COLUMNS_TO_IGNORE)

0 rows were completely blank and were removed.

0 columns were completely blank and were removed.

No blank columns or rows were found. Returning the original dataframe.



### **Characterizing the categorical variables**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

TIMESTAMP_TAG_COLUMN = 'date'
# TIMESTAMP_TAG_COLUMN: name (header) of the column containing the timestamps. 
# Keep TIMESTAMP_TAG_COLUMN = None if the dataframe do not contain timestamps.

# Dataframe with summary from the categorical variables returned as cat_vars_summary. 
# Simply modify this object on the left of equality:
cat_vars_summary = ewf.characterize_categorical_variables (df = DATASET, timestamp_tag_column = TIMESTAMP_TAG_COLUMN)



Finished analyzing the categorical variables. Check the summary dataframe:



,value,counts_of_occurences,percent_of_occurences,categorical_variable
0,NaN,0,0.000000,WeekStatus
1,Weekday,25056,71.506849,WeekStatus
2,Weekend,9984,28.493151,WeekStatus
3,NaN,0,0.000000,Day_of_week
4,Monday,5088,14.520548,Day_of_week
5,Tuesday,4992,14.246575,Day_of_week
6,Wednesday,4992,14.246575,Day_of_week
7,Thursday,4992,14.246575,Day_of_week
8,Friday,4992,14.246575,Day_of_week
9,Saturday,4992,14.246575,Day_of_week


Let's check the minimum and maximum dates:


In [ ]:
DATASET = dataset.copy(deep = True)
DATASET['date'] = DATASET['date'].astype(np.datetime64)
print(DATASET['date'].min())
print(DATASET['date'].max())

2018-01-01 00:00:00
2018-12-31 23:45:00


There is a single year of data. Let's group the dataframe by hour, so that we have equally-spaced data.

### **Grouping the dataframe by a timestamp**
- Numeric variables aggregated in terms of a custom function, passed as `aggregation_function`;
- Categorical variables aggregated in terms of mode, the most common value observed (maximum of the statistical distribution).

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be grouped

TIMESTAMP_TAG_COLUMN = "date"
#Alternatively: string (inside quotes) containing the name (header) of the timestamp column

SUBSET_OF_COLUMNS_TO_AGGREGATE = None
# SUBSET_OF_COLUMNS_TO_AGGREGATE: list of strings (inside quotes) containing the names 
# of the columns that will be aggregated. Use this argument if you want to aggregate only a subset,
# not the whole dataframe. Declare as a list even if there is a single column to group by.
# e.g. SUBSET_OF_COLUMNS_TO_AGGREGATE = ["response_feature"] will return the column 
# 'response_feature' grouped. SUBSET_OF_COLUMNS_TO_AGGREGATE = ["col1", 'col2'] will return columns
# 'col1' and 'col2' grouped.
# If you want to aggregate the whole subset, keep SUBSET_OF_COLUMNS_TO_AGGREGATE = None.

GROUPING_FREQUENCY_UNIT = 'hour'
#Alternatively: 'year', 'month', 'week', 'hour', 'minute', 'day', or 'second'

NUMBER_OF_PERIODS_TO_GROUP = 6
# Group by every NUMBER_OF_PERIODS_TO_GROUP = 1 periods (every day, if 'day' is selected).
#Bin size. Alternatively: any integer number. Check the instructions in function comments.

AGGREGATE_FUNCTION = 'mean'
# Keep the method inside quotes.
# Alternatively: use 'mean','sum', median','std', 'count', 'min','max','mode','geometric_mean',
# 'harmonic_mean','kurtosis','skew','geometric_std','interquartile_range','mean_standard_error',
# or 'entropy'

# ADJUST OF GROUPING BASED ON A FIXED TIMESTAMP
# You can specify the origin (start_time) or the offset (offset_time), which are equivalent.
# WARNING: DECLARE ONLY ONE OF THESE PARAMETERS. DO NOT DECLARE AN OFFSET IF AN ORIGIN WAS 
# SPECIFIED, AND VICE-VERSA.
START_TIME = None
OFFSET_TIME = None
# Alternatively, these parameters should be declared as a pandas Timestamp or in the
# specific notation of Pandas offset_time for the Grouper class:
# START_TIME = pd.Timestamp('2000-10-01 23:30:00', unit = 'ns')
# Simply substitute the Timestamp inside quotes by the correct start timestamp.
# This timestamp do not have to be complete, but must be interpretable by the Timestamp
# function.
# OFFSET_TIME = '23h30min', OFFSET_TIME = '2min', etc. Simply substitute the offset time
# inside quotes by the correct value.
# For examples on the notation for start and offset time, check Pandas grouper class
# documentation, and Pandas timestamp class documentation:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html

ADD_SUFFIX_TO_AGGREGATED_COLUMN = True
# ADD_SUFFIX_TO_AGGREGATED_COLUMN = True will add a suffix to the
# aggregated column. e.g. 'responseVar_mean'. If ADD_SUFFIX_TO_AGGREGATED_COLUMN
# = False, the aggregated column will have the original column name.
SUFFIX = None
# suffix = None. Keep it None if no suffix should be added, or if
# the name of the aggregate function should be used as suffix, after
# "_". Alternatively, set it as a string. As recommendation, put the
# "_" sign in the beginning of this string to separate the suffix from
# the original column name. e.g. if the response variable is 'Y' and
# suffix = '_agg', the new aggregated column will be named as 'Y_agg'


# New dataframe saved as grouped_df. 
# Simply modify this object on the left of equality:
dataset = ewf.GROUP_VARIABLES_BY_TIMESTAMP (df = DATASET, timestamp_tag_column = TIMESTAMP_TAG_COLUMN, subset_of_columns_to_aggregate = SUBSET_OF_COLUMNS_TO_AGGREGATE, grouping_frequency_unit = GROUPING_FREQUENCY_UNIT, number_of_periods_to_group = NUMBER_OF_PERIODS_TO_GROUP, aggregate_function = AGGREGATE_FUNCTION, start_time = START_TIME, offset_time = OFFSET_TIME, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COLUMN, suffix = SUFFIX)


Numerical variables of the dataframe grouped in terms of mean by every 6 H.

Categorical variables of the dataframe grouped in terms of 'mode' by every 6 H.

The mode is the most common value observed (maximum of the statistical distribution) for the categorical variable when we group data in terms of 6 H.

Dataframe successfully grouped. Check its 10 first rows (without the categorical/object variables):



,timestamp_grouped,Usage_kWh_mean,Lagging_Current_Reactive.Power_kVarh_mean,Leading_Current_Reactive_Power_kVarh_mean,CO2(tCO2)_mean,Lagging_Current_Power_Factor_mean,Leading_Current_Power_Factor_mean,NSM_mean,WeekStatus_mode,Day_of_week_mode,Load_Type_mode
0,2018-01-01 00:00:00,3.498333,3.986667,0.000000,0.000000,66.150417,100.000000,10350.0,Weekday,Monday,Light_Load
1,2018-01-01 06:00:00,3.640833,1.994583,7.832917,0.000000,82.951250,64.299167,31950.0,Weekday,Monday,Light_Load
2,2018-01-01 12:00:00,3.833750,0.000000,18.429167,0.000000,100.000000,21.776667,53550.0,Weekday,Monday,Light_Load
3,2018-01-01 18:00:00,3.687917,0.961250,13.277917,0.000000,92.095833,42.228750,75150.0,Weekday,Monday,Light_Load
4,2018-01-02 00:00:00,26.022917,15.584167,0.000000,0.010417,77.164583,100.000000,10350.0,Weekday,Thursday,Light_Load
5,2018-01-02 06:00:00,68.396250,33.150833,0.001667,0.030000,84.316667,100.000000,31950.0,Weekday,Thursday,Light_Load
6,2018-01-02 12:00:00,77.831667,34.658750,4.141667,0.034167,92.470000,91.563750,53550.0,Weekday,Thursday,Medium_Load
7,2018-01-02 18:00:00,82.526667,33.745417,0.352083,0.038750,92.103333,99.984583,75150.0,Weekday,Thursday,Maximum_Load
8,2018-01-03 00:00:00,3.940000,4.951250,0.000000,0.000000,62.464167,100.000000,10350.0,Weekday,Thursday,Light_Load
9,2018-01-03 06:00:00,32.161250,11.874583,0.682500,0.013750,80.681250,99.920417,31950.0,Weekday,Thursday,Light_Load


In [ ]:
dataset['Load_Type_mode'].unique()

array(['Light_Load', 'Medium_Load', 'Maximum_Load'], dtype=object)

### **Grouping the dataframe by a given variable**
- Categorical variables are grouped by this function only when a proper aggregation function is selected, like the 'mode'.
- If other aggregate is selected, only numeric variables are grouped.

In [ ]:
cat_vars_summary['categorical_variable'].unique()

array(['WeekStatus', 'Day_of_week', 'Load_Type'], dtype=object)

#### Case 1: return a statistics summary dataframe

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

VARIABLE_TO_GROUP_BY = 'Load_Type_mode'
# of the column in terms of which the dataframe will be grouped by. e.g. 
# VARIABLE_TO_GROUP_BY = "column1" will group the dataframe in terms of 'column1'.
# WARNING: do not use this function to group a dataframe in terms of a timestamp. To group by
# a timestamp, use function GROUP_VARIABLES_BY_TIMESTAMP instead.

RETURN_SUMMARY_DATAFRAME = True
# RETURN_SUMMARY_DATAFRAME = False. Set RETURN_SUMMARY_DATAFRAME = True if you want the function
# to return a dataframe containing summary statistics (obtained with the describe method).

SUBSET_OF_COLUMNS_TO_AGGREGATE = None
# SUBSET_OF_COLUMNS_TO_AGGREGATE: list of strings (inside quotes) containing the names 
# of the columns that will be aggregated. Use this argument if you want to aggregate only a subset,
# not the whole dataframe. Declare as a list even if there is a single column to group by.
# e.g. SUBSET_OF_COLUMNS_TO_AGGREGATE = ["response_feature"] will return the column 
# 'response_feature' grouped. SUBSET_OF_COLUMNS_TO_AGGREGATE = ["col1", 'col2'] will return columns
# 'col1' and 'col2' grouped.
# If you want to aggregate the whole subset, keep SUBSET_OF_COLUMNS_TO_AGGREGATE = None.

AGGREGATE_FUNCTION = 'mean'
# AGGREGATE_FUNCTION = 'mean': String defining the aggregation 
# method that will be applied. Possible values:
# 'median', 'mean', 'mode', 'sum', 'min', 'max', 'variance', 'count',
# 'standard_deviation', 'cum_sum', 'cum_prod', 'cum_max', 'cum_min',
# '10_percent_quantile', '20_percent_quantile',
# '25_percent_quantile', '30_percent_quantile', '40_percent_quantile',
# '50_percent_quantile', '60_percent_quantile', '70_percent_quantile',
# '75_percent_quantile', '80_percent_quantile', '90_percent_quantile',
# '95_percent_quantile', 'kurtosis', 'skew', 'interquartile_range',
# 'mean_standard_error', 'entropy'
# To use another aggregate function, you can use the .agg method, passing 
# the aggregate as argument, such as in:
# .agg(scipy.stats.mode), 
# where the argument is a Scipy aggregate function.
# If None or an invalid function is input, 'mean' will be used.

ADD_SUFFIX_TO_AGGREGATED_COLUMN = True
# ADD_SUFFIX_TO_AGGREGATED_COLUMN = True will add a suffix to the
# aggregated column. e.g. 'responseVar_mean'. If ADD_SUFFIX_TO_AGGREGATED_COLUMN
# = False, the aggregated column will have the original column name.
SUFFIX = None
# suffix = None. Keep it None if no suffix should be added, or if
# the name of the aggregate function should be used as suffix, after
# "_". Alternatively, set it as a string. As recommendation, put the
# "_" sign in the beginning of this string to separate the suffix from
# the original column name. e.g. if the response variable is 'Y' and
# suffix = '_agg', the new aggregated column will be named as 'Y_agg'


# Grouped dataframe, and summary statistics dataframe returned as:
# grouped_df and summary_agg_df, respectively.
# Simply modify these objects on the left of equality:
grouped_df, summary_agg_df = ewf.GROUP_DATAFRAME_BY_VARIABLE (df = DATASET, variable_to_group_by = VARIABLE_TO_GROUP_BY, return_summary_dataframe = RETURN_SUMMARY_DATAFRAME, subset_of_columns_to_aggregate = SUBSET_OF_COLUMNS_TO_AGGREGATE, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COLUMN, suffix = SUFFIX)


Numeric aggregate selected. Categorical variables will be aggregated in terms of mode, the most common value.

Dataframe successfully grouped. Check its 10 first rows:



,Load_Type_mode,Usage_kWh_mean_mean,Lagging_Current_Reactive.Power_kVarh_mean_mean,Leading_Current_Reactive_Power_kVarh_mean_mean,CO2(tCO2)_mean_mean,Lagging_Current_Power_Factor_mean_mean,Leading_Current_Power_Factor_mean_mean,NSM_mean_mean,timestamp_grouped_mode,WeekStatus_mode_mode,Day_of_week_mode_mode
0,Light_Load,18.249415,10.913750,2.669574,0.007186,73.291902,87.068042,27422.599532,2018-01-01 00:00:00,Weekday,Sunday
1,Maximum_Load,45.365428,18.221317,4.953830,0.020157,92.652093,83.793826,60322.277228,2018-01-02 18:00:00,Weekday,Friday
2,Medium_Load,35.162170,13.829237,6.174120,0.015120,89.039913,77.331528,68377.722772,2018-01-02 12:00:00,Weekday,Friday




Check the summary statistics dataframe, that is also being returned:



Usage_kWh_mean                                                        \
           count       mean        std       min        25%        50%   
0          854.0  18.249415  20.724999  2.615000   3.102187   3.737292   
1          303.0  45.365428  21.021234  3.330000  32.938333  46.598750   
2          303.0  35.162170  24.137978  2.924583  20.520000  30.539167   

                        Lagging_Current_Reactive.Power_kVarh_mean             \
         75%        max                                     count       mean   
0  36.442604  82.013333                                     854.0  10.913750   
1  58.099167  93.174583                                     303.0  18.221317   
2  49.248750  99.540417                                     303.0  13.829237   

   ... Leading_Current_Power_Factor_mean             NSM_mean                \
   ...                               75%         max    count          mean   
0  ...                        100.000000  100.000000    854.0  27422.599532   
1  ...                         89.658750  100.000000    303.0  60322.277228   
2  ...                         88.721042   99.999583    303.0  68377.722772   

                                                              
            std      min      25%      50%      75%      max  
0  18672.950603  10350.0  10350.0  31950.0  31950.0  75150.0  
1  10037.427827  53550.0  53550.0  53550.0  75150.0  75150.0  
2  10037.427827  53550.0  53550.0  75150.0  75150.0  75150.0  

[3 rows x 56 columns]

### **Creating columns with isolated information from the timestamps**
- Columns containing isolated information from the timestamp (each column with a given information): 
    - Values of year, month, week, day, hour, minute, or second may be extracted.

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

TIMESTAMP_TAG_COLUMN = "timestamp_grouped"
#Alternatively: string (inside quotes) containing the name (header) of the timestamp column
#Keep inside quotes.

LIST_OF_INFO_TO_EXTRACT = ['year', 'month', 'week', 'day']
# LIST_OF_INFO_TO_EXTRACT: list of information to extract from the timestamp. Each information
# will be extracted as a separate column. The allowed values are:
# 'year', 'month', 'week', 'day', 'hour', 'minute', or 'second'. Declare as a list even if only
# one information is going to be extracted. For instance:
# LIST_OF_INFO_TO_EXTRACT = ['second'] extracts only the second.
# LIST_OF_INFO_TO_EXTRACT = ['year', 'month', 'week', 'day'] extracts year, month, week and day. 

LIST_OF_NEW_COLUMN_NAMES = None
# list_of_new_column_names: list of names (strings) of the new created columns. 
# If no value is provided, it will be equals to extracted_info. For instance: if
# list_of_info_to_extract = ['year', 'month', 'week', 'day'] and list_of_new_column_names = None,
# the new columns will be named as 'year', 'month', 'week', and 'day'.
# WARNING: This list must contain the same number of elements of list_of_info_to_extract and both
# must be in the same order. Considering the same example of list, if list_of_new_column_names =
# ['col1', 'col2', 'col3', 'col4'], 'col1' will be referrent to 'year', 'col2' to 'month', 'col3'
# to 'week', and 'col4' to 'day'

# New dataframe saved as new_df. Simply modify this object on the left of equality:
dataset = ewf.EXTRACT_TIMESTAMP_INFO (df = DATASET, timestamp_tag_column = TIMESTAMP_TAG_COLUMN, list_of_info_to_extract = LIST_OF_INFO_TO_EXTRACT, list_of_new_column_names = LIST_OF_NEW_COLUMN_NAMES)

Timestamp information successfully extracted. Check dataset's 10 first rows:



,timestamp_grouped,Usage_kWh_mean,Lagging_Current_Reactive.Power_kVarh_mean,Leading_Current_Reactive_Power_kVarh_mean,CO2(tCO2)_mean,Lagging_Current_Power_Factor_mean,Leading_Current_Power_Factor_mean,NSM_mean,WeekStatus_mode,Day_of_week_mode,Load_Type_mode,year,month,week,day
0,2018-01-01 00:00:00,3.498333,3.986667,0.000000,0.000000,66.150417,100.000000,10350.0,Weekday,Monday,Light_Load,2018,1,1,1
1,2018-01-01 06:00:00,3.640833,1.994583,7.832917,0.000000,82.951250,64.299167,31950.0,Weekday,Monday,Light_Load,2018,1,1,1
2,2018-01-01 12:00:00,3.833750,0.000000,18.429167,0.000000,100.000000,21.776667,53550.0,Weekday,Monday,Light_Load,2018,1,1,1
3,2018-01-01 18:00:00,3.687917,0.961250,13.277917,0.000000,92.095833,42.228750,75150.0,Weekday,Monday,Light_Load,2018,1,1,1
4,2018-01-02 00:00:00,26.022917,15.584167,0.000000,0.010417,77.164583,100.000000,10350.0,Weekday,Thursday,Light_Load,2018,1,1,2
5,2018-01-02 06:00:00,68.396250,33.150833,0.001667,0.030000,84.316667,100.000000,31950.0,Weekday,Thursday,Light_Load,2018,1,1,2
6,2018-01-02 12:00:00,77.831667,34.658750,4.141667,0.034167,92.470000,91.563750,53550.0,Weekday,Thursday,Medium_Load,2018,1,1,2
7,2018-01-02 18:00:00,82.526667,33.745417,0.352083,0.038750,92.103333,99.984583,75150.0,Weekday,Thursday,Maximum_Load,2018,1,1,2
8,2018-01-03 00:00:00,3.940000,4.951250,0.000000,0.000000,62.464167,100.000000,10350.0,Weekday,Thursday,Light_Load,2018,1,1,3
9,2018-01-03 06:00:00,32.161250,11.874583,0.682500,0.013750,80.681250,99.920417,31950.0,Weekday,Thursday,Light_Load,2018,1,1,3


### **Calculating differences between successive timestamps (delays)**

#### Case 1: return average delay

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

TIMESTAMP_TAG_COLUMN = "timestamp_grouped"
# "timestamp_grouped" is the column created by the function which aggregates the timestamps.
# Alternatively: string (inside quotes) containing the name (header) of the timestamp column
# on the left (from which the right timestamp will be subtracted).
# Keep inside quotes.

NEW_TIMEDELTA_COLUMN_NAME = None
# Name of the new column. If no value is provided, the default name 
# [timestamp_tag_column1]-[timestamp_tag_column2] will be given.
# Alternatively: keep it as None or input a name (string) for the new column inside quotes:
# e.g. NEW_TIMEDELTA_COLUMN_NAME = "Timestamp_difference"
    
RETURNED_TIMEDELTA_UNIT = 'day'
# Unit of the new column. If no value is provided, the unit will be considered as nanoseconds. 
# Alternatively: keep it None, for the results in nanoseconds, or input RETURNED_TIMEDELTA_UNIT = 
# 'year', 'month', 'day', 'hour', 'minute', or 'second' (keep these inside quotes).

RETURN_AVG_DELAY = True
# RETURN_AVG_DELAY = True will print and return the value of the average delay.
# RETURN_AVG_DELAY = False will omit this information

# New dataframe saved as new_df. Simply modify this object on the left of equality.
# Average delay float value istored into variable avg_delay. 
# Simply modify this object on the left of equality.
dataset_delay, avg_delay = ewf.CALCULATE_DELAY (df = DATASET, timestamp_tag_column = TIMESTAMP_TAG_COLUMN, new_timedelta_column_name  = NEW_TIMEDELTA_COLUMN_NAME, returned_timedelta_unit = RETURNED_TIMEDELTA_UNIT, return_avg_delay = RETURN_AVG_DELAY)

Returned timedelta in days.

Time delays successfully calculated. Check dataset's 10 first rows:



,timestamp_grouped,Usage_kWh_mean,Lagging_Current_Reactive.Power_kVarh_mean,Leading_Current_Reactive_Power_kVarh_mean,CO2(tCO2)_mean,Lagging_Current_Power_Factor_mean,Leading_Current_Power_Factor_mean,NSM_mean,WeekStatus_mode,Day_of_week_mode,Load_Type_mode,year,month,week,day,timestamp_grouped_delayed,time_delay_day
0,2018-01-01 00:00:00,3.498333,3.986667,0.000000,0.000000,66.150417,100.000000,10350.0,Weekday,Monday,Light_Load,2018,1,1,1,2018-01-01 06:00:00,0.25
1,2018-01-01 06:00:00,3.640833,1.994583,7.832917,0.000000,82.951250,64.299167,31950.0,Weekday,Monday,Light_Load,2018,1,1,1,2018-01-01 12:00:00,0.25
2,2018-01-01 12:00:00,3.833750,0.000000,18.429167,0.000000,100.000000,21.776667,53550.0,Weekday,Monday,Light_Load,2018,1,1,1,2018-01-01 18:00:00,0.25
3,2018-01-01 18:00:00,3.687917,0.961250,13.277917,0.000000,92.095833,42.228750,75150.0,Weekday,Monday,Light_Load,2018,1,1,1,2018-01-02 00:00:00,0.25
4,2018-01-02 00:00:00,26.022917,15.584167,0.000000,0.010417,77.164583,100.000000,10350.0,Weekday,Thursday,Light_Load,2018,1,1,2,2018-01-02 06:00:00,0.25
5,2018-01-02 06:00:00,68.396250,33.150833,0.001667,0.030000,84.316667,100.000000,31950.0,Weekday,Thursday,Light_Load,2018,1,1,2,2018-01-02 12:00:00,0.25
6,2018-01-02 12:00:00,77.831667,34.658750,4.141667,0.034167,92.470000,91.563750,53550.0,Weekday,Thursday,Medium_Load,2018,1,1,2,2018-01-02 18:00:00,0.25
7,2018-01-02 18:00:00,82.526667,33.745417,0.352083,0.038750,92.103333,99.984583,75150.0,Weekday,Thursday,Maximum_Load,2018,1,1,2,2018-01-03 00:00:00,0.25
8,2018-01-03 00:00:00,3.940000,4.951250,0.000000,0.000000,62.464167,100.000000,10350.0,Weekday,Thursday,Light_Load,2018,1,1,3,2018-01-03 06:00:00,0.25
9,2018-01-03 06:00:00,32.161250,11.874583,0.682500,0.013750,80.681250,99.920417,31950.0,Weekday,Thursday,Light_Load,2018,1,1,3,2018-01-03 12:00:00,0.25


Average delay = 0.25 day



In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

#New dataframes saved as df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values.
# Simply modify this object on the left of equality:
df_shape, df_columns_array, df_dtypes, df_general_statistics, df_missing_values = ewf.df_general_characterization (df = DATASET)

Dataframe's 10 first rows:



,timestamp_grouped,Usage_kWh_mean,Lagging_Current_Reactive.Power_kVarh_mean,Leading_Current_Reactive_Power_kVarh_mean,CO2(tCO2)_mean,Lagging_Current_Power_Factor_mean,Leading_Current_Power_Factor_mean,NSM_mean,WeekStatus_mode,Day_of_week_mode,Load_Type_mode,year,month,week,day
0,2018-01-01 00:00:00,3.498333,3.986667,0.000000,0.000000,66.150417,100.000000,10350.0,Weekday,Monday,Light_Load,2018,1,1,1
1,2018-01-01 06:00:00,3.640833,1.994583,7.832917,0.000000,82.951250,64.299167,31950.0,Weekday,Monday,Light_Load,2018,1,1,1
2,2018-01-01 12:00:00,3.833750,0.000000,18.429167,0.000000,100.000000,21.776667,53550.0,Weekday,Monday,Light_Load,2018,1,1,1
3,2018-01-01 18:00:00,3.687917,0.961250,13.277917,0.000000,92.095833,42.228750,75150.0,Weekday,Monday,Light_Load,2018,1,1,1
4,2018-01-02 00:00:00,26.022917,15.584167,0.000000,0.010417,77.164583,100.000000,10350.0,Weekday,Thursday,Light_Load,2018,1,1,2
5,2018-01-02 06:00:00,68.396250,33.150833,0.001667,0.030000,84.316667,100.000000,31950.0,Weekday,Thursday,Light_Load,2018,1,1,2
6,2018-01-02 12:00:00,77.831667,34.658750,4.141667,0.034167,92.470000,91.563750,53550.0,Weekday,Thursday,Medium_Load,2018,1,1,2
7,2018-01-02 18:00:00,82.526667,33.745417,0.352083,0.038750,92.103333,99.984583,75150.0,Weekday,Thursday,Maximum_Load,2018,1,1,2
8,2018-01-03 00:00:00,3.940000,4.951250,0.000000,0.000000,62.464167,100.000000,10350.0,Weekday,Thursday,Light_Load,2018,1,1,3
9,2018-01-03 06:00:00,32.161250,11.874583,0.682500,0.013750,80.681250,99.920417,31950.0,Weekday,Thursday,Light_Load,2018,1,1,3




Dataframe's 10 last rows:



,timestamp_grouped,Usage_kWh_mean,Lagging_Current_Reactive.Power_kVarh_mean,Leading_Current_Reactive_Power_kVarh_mean,CO2(tCO2)_mean,Lagging_Current_Power_Factor_mean,Leading_Current_Power_Factor_mean,NSM_mean,WeekStatus_mode,Day_of_week_mode,Load_Type_mode,year,month,week,day
1450,2018-12-29 12:00:00,3.738750,0.000000,17.347500,0.0,100.000000,21.450417,53550.0,Weekend,Saturday,Light_Load,2018,12,52,29
1451,2018-12-29 18:00:00,3.503750,0.741667,11.107917,0.0,93.527917,46.187500,75150.0,Weekend,Saturday,Light_Load,2018,12,52,29
1452,2018-12-30 00:00:00,3.437500,2.937083,0.017083,0.0,76.224583,99.980833,10350.0,Weekend,Sunday,Light_Load,2018,12,52,30
1453,2018-12-30 06:00:00,3.632083,1.492917,8.236667,0.0,88.201250,60.188333,31950.0,Weekend,Sunday,Light_Load,2018,12,52,30
1454,2018-12-30 12:00:00,3.585000,0.000000,15.943750,0.0,100.000000,22.270833,53550.0,Weekend,Sunday,Light_Load,2018,12,52,30
1455,2018-12-30 18:00:00,3.478750,0.600000,11.790000,0.0,94.955000,44.816250,75150.0,Weekend,Sunday,Light_Load,2018,12,52,30
1456,2018-12-31 00:00:00,3.417500,2.526667,0.122917,0.0,80.582917,99.920417,10350.0,Weekday,Monday,Light_Load,2018,12,1,31
1457,2018-12-31 06:00:00,3.387917,1.108333,8.450417,0.0,91.627917,58.542917,31950.0,Weekday,Monday,Light_Load,2018,12,1,31
1458,2018-12-31 12:00:00,3.627083,0.000000,16.083750,0.0,100.000000,22.386250,53550.0,Weekday,Monday,Light_Load,2018,12,1,31
1459,2018-12-31 18:00:00,3.695833,1.132500,10.553333,0.0,91.022083,47.307083,75150.0,Weekday,Monday,Light_Load,2018,12,1,31




Dataframe's shape = (number of rows, number of columns) =



...

dataframe_column
timestamp_grouped                            datetime64[ns]
Usage_kWh_mean                                      float64
Lagging_Current_Reactive.Power_kVarh_mean           float64
Leading_Current_Reactive_Power_kVarh_mean           float64
CO2(tCO2)_mean                                      float64
Lagging_Current_Power_Factor_mean                   float64
Leading_Current_Power_Factor_mean                   float64
NSM_mean                                            float64
WeekStatus_mode                                      object
Day_of_week_mode                                     object
Load_Type_mode                                       object
year                                                  int64
month                                                 int64
week                                                 UInt32
day                                                   int64
Name: dtype_series, dtype: object



Dataframe's general (summary) statistics for numeric variables:



dataframe_column,Usage_kWh_mean,Lagging_Current_Reactive.Power_kVarh_mean,Leading_Current_Reactive_Power_kVarh_mean,CO2(tCO2)_mean,Lagging_Current_Power_Factor_mean,Leading_Current_Power_Factor_mean,NSM_mean,year,month,week,day
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.0,1460.000000,1460.000000,1460.000000
mean,27.386892,13.035384,3.870949,0.011524,80.578056,84.367870,42750.000000,2018.0,6.526027,26.430137,15.720548
std,24.325781,11.148181,4.944305,0.011630,14.676147,22.087120,24157.808796,0.0,3.449033,15.052066,8.799261
min,2.615000,0.000000,0.000000,0.000000,42.647500,16.917500,10350.000000,2018.0,1.000000,1.000000,1.000000
25%,3.422500,3.488125,0.000000,0.000000,73.359167,81.641771,26550.000000,2018.0,4.000000,13.000000,8.000000
50%,25.332083,8.995833,2.651875,0.010833,84.320625,91.208542,42750.000000,2018.0,7.000000,26.000000,16.000000
75%,45.710729,22.799271,5.654792,0.020417,92.017917,100.000000,58950.000000,2018.0,10.000000,39.000000,23.000000
max,99.540417,43.711250,24.723333,0.045417,100.000000,100.000000,75150.000000,2018.0,12.000000,52.000000,31.000000




Missing values on each feature; and missingness considering all rows from the dataframe:
(note: 'missingness_accross_rows' was calculated by: checking which rows have at least one missing value (NA); and then comparing total rows with NAs with total rows in the dataframe).



,count_of_missing_values,proportion_of_missing_values,percent_of_missing_values
dataframe_column,,,
timestamp_grouped,0,0.0,0.0
Usage_kWh_mean,0,0.0,0.0
Lagging_Current_Reactive.Power_kVarh_mean,0,0.0,0.0
Leading_Current_Reactive_Power_kVarh_mean,0,0.0,0.0
CO2(tCO2)_mean,0,0.0,0.0
Lagging_Current_Power_Factor_mean,0,0.0,0.0
Leading_Current_Power_Factor_mean,0,0.0,0.0
NSM_mean,0,0.0,0.0
WeekStatus_mode,0,0.0,0.0


## **Exporting the dataframe as CSV file (to notebook's workspace)**

In [ ]:
## WARNING: all files exported from this function are .csv (comma separated values)

DATAFRAME_OBJ_TO_BE_EXPORTED = dataset
# Alternatively: object containing the dataset to be exported.
# DATAFRAME_OBJ_TO_BE_EXPORTED: dataframe object that is going to be exported from the
# function. Since it is an object (not a string), it should not be declared in quotes.
# example: DATAFRAME_OBJ_TO_BE_EXPORTED = dataset will export the dataset object.
# ATTENTION: The dataframe object must be a Pandas dataframe.

FILE_DIRECTORY_PATH = ""
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "" 
# or FILE_DIRECTORY_PATH = "folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None

NEW_FILE_NAME_WITHOUT_EXTENSION = "steel_industry_grouped"
# NEW_FILE_NAME_WITHOUT_EXTENSION - (string, in quotes): input the name of the 
# file without the extension. e.g. set NEW_FILE_NAME_WITHOUT_EXTENSION = "my_file" 
# to export the CSV file 'my_file.csv' to notebook's workspace.

idsw.export_pd_dataframe_as_csv (dataframe_obj_to_be_exported = DATAFRAME_OBJ_TO_BE_EXPORTED, new_file_name_without_extension = NEW_FILE_NAME_WITHOUT_EXTENSION, file_directory_path = FILE_DIRECTORY_PATH)

Dataframe steel_industry_grouped exported as CSV file to notebook's workspace as 'steel_industry_grouped.csv'.


## **Downloading a file from Google Colab to the local machine; or uploading a file from the machine to Colab's instant memory**

#### Case 2: download a file from Colab's workspace

In [ ]:
ACTION = 'download'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

FILE_TO_DOWNLOAD_FROM_COLAB = 'steel_industry_grouped.csv'
# FILE_TO_DOWNLOAD_FROM_COLAB = None. This parameter is obbligatory when
# action = 'download'. 
# Declare as FILE_TO_DOWNLOAD_FROM_COLAB the file that you want to download, with
# the correspondent extension.
# It should not be declared in quotes.
# e.g. to download a dictionary named dict, FILE_TO_DOWNLOAD_FROM_COLAB = 'dict.pkl'
# To download a dataframe named df, declare FILE_TO_DOWNLOAD_FROM_COLAB = 'df.csv'
# To export a model nameACTION = 'upload'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

idsw.upload_to_or_download_file_from_colab (action = ACTION, file_to_download_from_colab = FILE_TO_DOWNLOAD_FROM_COLAB)

The file will be downloaded to your computer.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File steel_industry_grouped.csv successfully downloaded from Colab environment.


****